In [91]:
import pandas as pd
from dotenv import load_dotenv

from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

from api_helpers.clients import get_postgres_client


load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"

)

pg = get_postgres_client()


In [ ]:
import pandas as pd
import numpy as np

df = pg.fetch_data(
            """     
            SELECT 
                s.unique_id,
                s.race_id,
                s.race_time,
                s.race_date,
                s.horse_id,
                s.horse_name,
                s.selection_type,
                s.market_type,
                s.stake_points,
                COALESCE(s.average_price_matched, s.requested_odds) AS betfair_sp,
                s.created_at,
                pd.finishing_position,
                pd.number_of_runners
            FROM live_betting.selections s 
            LEFT JOIN public.unioned_results_data pd
                ON s.horse_id = pd.horse_id
                AND s.race_id = pd.race_id
            WHERE s.valid = true
            AND s.market_id = 'feedback'
            ORDER BY s.created_at;
            """
)


def add_results_booleans(df):
    """
    Add win and place boolean columns based on finishing position and number of runners.
    Equivalent to the results_with_booleans CTE.
    """
    df = df.copy()

    # Win boolean: true if finishing_position = '1'
    df["win"] = df["finishing_position"] == "1"

    # Place boolean based on number of runners
    def calculate_placed(row):
        pos = row["finishing_position"]
        runners = row["number_of_runners"]

        # Convert position to int for comparison
        try:
            pos_int = int(pos)
        except (ValueError, TypeError):
            return False

        # Less than 8 runners: places 1-2
        if runners < 8:
            return pos_int in [1, 2]
        # 8-15 runners: places 1-3
        elif 8 <= runners <= 15:
            return pos_int in [1, 2, 3]
        # 16+ runners: places 1-4
        elif runners >= 16:
            return pos_int in [1, 2, 3, 4]
        else:
            return False

    df["placed"] = df.apply(calculate_placed, axis=1)

    return df


def calculate_profit_loss(df):
    """
    Calculate profit/loss for each bet, both regular and stake-points weighted.
    Equivalent to the profit_loss_calc CTE.
    """
    df = df.copy()

    def calc_pnl(row):
        selection_type = row["selection_type"]
        market_type = row["market_type"]
        betfair_sp = row["betfair_sp"]
        win = row["win"]
        placed = row["placed"]
        stake_points = row["stake_points"]
        odds_returned = betfair_sp - 1
        multiplier = 0.8  # 80% return on win bets

        # Regular profit/loss calculation
        if selection_type == "BACK" and market_type == "WIN":
            profit_loss = multiplier * odds_returned if win else -1
            profit_loss_stake_points = (
                stake_points * multiplier * odds_returned if win else -stake_points
            )

        elif selection_type == "BACK" and market_type == "PLACE":
            profit_loss = multiplier * odds_returned if placed else -1
            profit_loss_stake_points = (
                stake_points * multiplier * odds_returned if placed else -stake_points
            )

        elif selection_type == "LAY" and market_type == "WIN":
            profit_loss = -odds_returned if win else multiplier
            profit_loss_stake_points = (
                -stake_points if win else stake_points * multiplier
            )

        elif selection_type == "LAY" and market_type == "PLACE":
            profit_loss = -odds_returned if placed else multiplier
            profit_loss_stake_points = (
                -stake_points if placed else stake_points * multiplier
            )

        else:
            profit_loss = 0

        # Stake calculation
        if selection_type == "BACK":
            level_stake = 1
            confidence_stake = stake_points
        elif selection_type == "LAY":
            level_stake = betfair_sp - 1
            confidence_stake = stake_points / (betfair_sp - 1)
        else:
            level_stake = 0
            confidence_stake = 0

        return pd.Series(
            {
                "profit_loss": profit_loss,
                "profit_loss_stake_points": profit_loss_stake_points,
                "level_stake": level_stake,
                "confidence_stake": confidence_stake,
            }
        )

    pnl_data = df.apply(calc_pnl, axis=1)
    df = pd.concat([df, pnl_data], axis=1)

    return df


def calculate_level_stakes_running_totals(df):
    """
    Calculate running totals and counts for different bet types.
    """
    df = df.copy()
    df = df.sort_values("created_at").reset_index(drop=True)

    # Create masks for different bet types
    back_win_mask = (df["selection_type"] == "BACK") & (df["market_type"] == "WIN")
    back_place_mask = (df["selection_type"] == "BACK") & (df["market_type"] == "PLACE")
    lay_win_mask = (df["selection_type"] == "LAY") & (df["market_type"] == "WIN")
    lay_place_mask = (df["selection_type"] == "LAY") & (df["market_type"] == "PLACE")

    # Running P&L totals
    df["running_total_back_win_pnl"] = (df["profit_loss"] * back_win_mask).cumsum()
    df["running_total_back_place_pnl"] = (df["profit_loss"] * back_place_mask).cumsum()
    df["running_total_lay_win_pnl"] = (df["profit_loss"] * lay_win_mask).cumsum()
    df["running_total_lay_place_pnl"] = (df["profit_loss"] * lay_place_mask).cumsum()
    df["running_total_all_bets_pnl"] = df["profit_loss"].cumsum()

    # Running stake totals
    df["running_stake_back_win"] = (df["level_stake"] * back_win_mask).cumsum()
    df["running_stake_back_place"] = (df["level_stake"] * back_place_mask).cumsum()
    df["running_stake_lay_win"] = (df["level_stake"] * lay_win_mask).cumsum()
    df["running_stake_lay_place"] = (df["level_stake"] * lay_place_mask).cumsum()
    df["running_stake_total"] = df["level_stake"].cumsum()

    # Running counts
    df["total_back_win_count"] = back_win_mask.cumsum()
    df["total_back_place_count"] = back_place_mask.cumsum()
    df["total_lay_win_count"] = lay_win_mask.cumsum()
    df["total_lay_place_count"] = lay_place_mask.cumsum()
    df["total_bet_count"] = range(1, len(df) + 1)

    return df


def calculate_weighted_stakes_running_totals(df):

    df = df.copy()
    df = df.sort_values("created_at").reset_index(drop=True)

    # Create masks for different bet types
    back_win_mask = (df["selection_type"] == "BACK") & (df["market_type"] == "WIN")
    back_place_mask = (df["selection_type"] == "BACK") & (df["market_type"] == "PLACE")
    lay_win_mask = (df["selection_type"] == "LAY") & (df["market_type"] == "WIN")
    lay_place_mask = (df["selection_type"] == "LAY") & (df["market_type"] == "PLACE")

    df["running_stake_points_back_win_pnl"] = (
        df["profit_loss_stake_points"] * back_win_mask
    ).cumsum()
    df["running_stake_points_back_place_pnl"] = (
        df["profit_loss_stake_points"] * back_place_mask
    ).cumsum()
    df["running_stake_points_lay_win_pnl"] = (
        df["profit_loss_stake_points"] * lay_win_mask
    ).cumsum()
    df["running_stake_points_lay_place_pnl"] = (
        df["profit_loss_stake_points"] * lay_place_mask
    ).cumsum()
    df["running_stake_points_total_pnl"] = df["profit_loss_stake_points"].cumsum()

    # Running stake-points totals
    df["running_stake_points_back_win"] = (
        df["confidence_stake"] * back_win_mask
    ).cumsum()
    df["running_stake_points_back_place"] = (
        df["confidence_stake"] * back_place_mask
    ).cumsum()
    df["running_stake_points_lay_win"] = (
        df["confidence_stake"] * lay_win_mask
    ).cumsum()
    df["running_stake_points_lay_place"] = (
        df["confidence_stake"] * lay_place_mask
    ).cumsum()
    df["running_stake_points_total"] = df["confidence_stake"].cumsum()

    return df


def calculate_level_stakes_roi_metrics(df):
    """
    Calculate ROI percentages for different bet types.
    Equivalent to the roi_calculations CTE.
    """
    df = df.copy()

    # Helper function to calculate ROI safely (avoid division by zero)
    def safe_roi(profit, stake):
        return np.where(stake > 0, (profit / stake) * 100, np.nan)

    # Running ROI calculations (as percentages)
    df["running_roi_back_win"] = safe_roi(
        df["running_total_back_win_pnl"], df["running_stake_back_win"]
    ).round(2)
    df["running_roi_back_place"] = safe_roi(
        df["running_total_back_place_pnl"], df["running_stake_back_place"]
    ).round(2)
    df["running_roi_lay_win"] = safe_roi(
        df["running_total_lay_win_pnl"], df["running_stake_lay_win"]
    ).round(2)
    df["running_roi_lay_place"] = safe_roi(
        df["running_total_lay_place_pnl"], df["running_stake_lay_place"]
    ).round(2)
    df["running_roi_overall"] = safe_roi(
        df["running_total_all_bets_pnl"], df["running_stake_total"]
    ).round(2)

    return df


def calculate_weighted_stakes_roi_metrics(df):
    """
    Calculate stake-points weighted ROI percentages for different bet types.
    """
    df = df.copy()

    # Helper function to calculate ROI safely (avoid division by zero)
    def safe_roi(profit, stake):
        return np.where(stake > 0, (profit / stake) * 100, np.nan)

    # Running stake-points weighted ROI calculations (as percentages)
    df["running_roi_stake_back_win"] = safe_roi(
        df["running_stake_points_back_win_pnl"], df["running_stake_points_back_win"]
    ).round(2)
    df["running_roi_stake_back_place"] = safe_roi(
        df["running_stake_points_back_place_pnl"], df["running_stake_points_back_place"]
    ).round(2)
    df["running_roi_stake_lay_win"] = safe_roi(
        df["running_stake_points_lay_win_pnl"], df["running_stake_points_lay_win"]
    ).round(2)
    df["running_roi_stake_lay_place"] = safe_roi(
        df["running_stake_points_lay_place_pnl"], df["running_stake_points_lay_place"]
    ).round(2)
    df["running_roi_overall_stake_points"] = safe_roi(
        df["running_stake_points_total_pnl"], df["running_stake_points_total"]
    ).round(2)

    return df


def process_betting_data(df):
    """
    Main function to process raw betting data through all calculation steps.
    """
    # Apply all transformations in sequence
    df = add_results_booleans(df)
    df = calculate_profit_loss(df)
    df = calculate_level_stakes_running_totals(df)
    df = calculate_weighted_stakes_running_totals(df)
    df = calculate_level_stakes_roi_metrics(df)
    df = calculate_weighted_stakes_roi_metrics(df)

    return df


# Process the data
tf = process_betting_data(df)

# # Select output columns (equivalent to final SELECT in original query)
# output_columns = [
#     'unique_id', 'created_at', 'stake_points', 'profit_loss', 'profit_loss_stake_points',
#     'running_total_back_win', 'running_total_back_place', 'running_total_lay_win',
#     'running_total_lay_place', 'running_total_all_bets', 'running_total_all_bets_stake_points',
#     'running_stake_total', 'running_stake_points_total', 'running_stake_weighted_total',
#     'running_roi_back_win', 'running_roi_back_place', 'running_roi_lay_win',
#     'running_roi_lay_place', 'running_roi_overall', 'running_roi_overall_stake_points',
#     'total_back_win_count', 'total_back_place_count', 'total_lay_win_count',
#     'total_lay_place_count', 'total_bet_count'
# ]

# df_final = df_processed[output_columns]

In [104]:
tf[
    [
        "unique_id",
        "race_id",
        "race_time",
        "race_date",
        "horse_id",
        "horse_name",
        "selection_type",
        "market_type",
        "stake_points",
        "betfair_sp",
        "created_at",
        "finishing_position",
        "number_of_runners",
        "win",
        "placed",
        "profit_loss",
        "profit_loss_stake_points",
        "level_stake",
        "confidence_stake",
        "running_total_back_win_pnl",
        "running_total_back_place_pnl",
        "running_total_lay_win_pnl",
        "running_total_lay_place_pnl",
        "running_total_all_bets_pnl",
        "running_stake_back_win",
        "running_stake_back_place",
        "running_stake_lay_win",
        "running_stake_lay_place",
        "running_stake_total",
        "total_back_win_count",
        "total_back_place_count",
        "total_lay_win_count",
        "total_lay_place_count",
        "total_bet_count",
        "running_stake_points_back_win_pnl",
        "running_stake_points_back_place_pnl",
        "running_stake_points_lay_win_pnl",
        "running_stake_points_lay_place_pnl",
        "running_stake_points_total_pnl",
        "running_stake_points_back_win",
        "running_stake_points_back_place",
        "running_stake_points_lay_win",
        "running_stake_points_lay_place",
        "running_stake_points_total",
        "running_roi_back_win",
        "running_roi_back_place",
        "running_roi_lay_win",
        "running_roi_lay_place",
        "running_roi_overall",
        "running_roi_stake_back_win",
        "running_roi_stake_back_place",
        "running_roi_stake_lay_win",
        "running_roi_stake_lay_place",
        "running_roi_overall_stake_points",
    ]
].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 54 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   unique_id                            6 non-null      object        
 1   race_id                              6 non-null      int64         
 2   race_time                            6 non-null      datetime64[ns]
 3   race_date                            6 non-null      object        
 4   horse_id                             6 non-null      int64         
 5   horse_name                           6 non-null      object        
 6   selection_type                       6 non-null      object        
 7   market_type                          6 non-null      object        
 8   stake_points                         6 non-null      float64       
 9   betfair_sp                           6 non-null      float64       
 10  created_at        

In [103]:
tf.to_csv('~/Desktop/betting_results_hack_output.csv', index=False)

In [51]:


df = df.copy()
df = df.sort_values("created_at").reset_index(drop=True)

# Create masks for different bet types
back_win_mask = (df["selection_type"] == "BACK") & (df["market_type"] == "WIN")
back_place_mask = (df["selection_type"] == "BACK") & (df["market_type"] == "PLACE")
lay_win_mask = (df["selection_type"] == "LAY") & (df["market_type"] == "WIN")
lay_place_mask = (df["selection_type"] == "LAY") & (df["market_type"] == "PLACE")

# # Running P&L totals
# df["running_total_back_win"] = (df["profit_loss"] * back_win_mask).cumsum()
# df["running_total_back_place"] = (df["profit_loss"] * back_place_mask).cumsum()
# df["running_total_lay_win"] = (df["profit_loss"] * lay_win_mask).cumsum()
# df["running_total_lay_place"] = (df["profit_loss"] * lay_place_mask).cumsum()
# df["running_total_all_bets"] = df["profit_loss"].cumsum()

# # Running stake-points totals
# df['running_stake_points_back_win'] = (df["profit_loss_stake_points"] * back_win_mask).cumsum()
# df['running_stake_points_back_place'] = (df["profit_loss_stake_points"] * back_place_mask).cumsum()
# df['running_stake_points_lay_win'] = (df["profit_loss_stake_points"] * lay_win_mask).cumsum()
# df['running_stake_points_lay_place'] = (df["profit_loss_stake_points"] * lay_place_mask).cumsum()
# df["running_stake_points_total"] = df["profit_loss_stake_points"].cumsum()


# # Running stake totals
# df["running_stake_back_win"] = (df["level_stake"] * back_win_mask).cumsum()
# df["running_stake_back_place"] = (df["level_stake"] * back_place_mask).cumsum()
# df["running_stake_lay_win"] = (df["level_stake"] * lay_win_mask).cumsum()
# df["running_stake_lay_place"] = (df["level_stake"] * lay_place_mask).cumsum()
# df["running_stake_total"] = df["level_stake"].cumsum()

# # Running stake-points totals
# df['running_stake_points_back_win'] = (df["confidence_stake"] * back_win_mask).cumsum()
# df['running_stake_points_back_place'] = (df["confidence_stake"] * back_place_mask).cumsum()
# df['running_stake_points_lay_win'] = (df["confidence_stake"] * lay_win_mask).cumsum()
# df['running_stake_points_lay_place'] = (df["confidence_stake"] * lay_place_mask).cumsum()
# df["running_stake_points_total"] = df["confidence_stake"].cumsum()

# # # For stake-points weighted ROI, we need running total of (stake * stake_points)
# # df["stake_weighted_by_points"] = df["level_stake"] * df["stake_points"]
# # df["running_stake_weighted_total"] = df["stake_weighted_by_points"].cumsum()

# # Running counts
# df["total_back_win_count"] = back_win_mask.cumsum()
# df["total_back_place_count"] = back_place_mask.cumsum()
# df["total_lay_win_count"] = lay_win_mask.cumsum()
# df["total_lay_place_count"] = lay_place_mask.cumsum()
# df["total_bet_count"] = range(1, len(df) + 1)



In [62]:
df.columns

Index(['unique_id', 'race_id', 'race_time', 'race_date', 'horse_id',
       'horse_name', 'selection_type', 'market_type', 'stake_points',
       'betfair_sp', 'created_at', 'finishing_position', 'number_of_runners',
       'win', 'placed', 'profit_loss', 'profit_loss_stake_points',
       'level_stake', 'confidence_stake'],
      dtype='object')

In [69]:
p = df[
    [
        "selection_type",
        "market_type",
        "stake_points",
        "betfair_sp",
        "win",
        "placed",
        "profit_loss",
        "profit_loss_stake_points",
        "level_stake",
        "confidence_stake",
    ]
].copy()

back_win_mask = (p["selection_type"] == "BACK") & (p["market_type"] == "WIN")
back_place_mask = (p["selection_type"] == "BACK") & (p["market_type"] == "PLACE")
lay_win_mask = (p["selection_type"] == "LAY") & (p["market_type"] == "WIN")
lay_place_mask = (p["selection_type"] == "LAY") & (p["market_type"] == "PLACE")

In [70]:
p['running_stake_points_back_win'] = (p["profit_loss_stake_points"] * back_win_mask).cumsum()
p['running_stake_points_back_place'] = (p["profit_loss_stake_points"] * back_place_mask).cumsum()
p['running_stake_points_lay_win'] = (p["profit_loss_stake_points"] * lay_win_mask).cumsum()
p['running_stake_points_lay_place'] = (p["profit_loss_stake_points"] * lay_place_mask).cumsum()
p["running_stake_points_total"] = p["profit_loss_stake_points"].cumsum()

In [83]:
p['running_stake_points_back_win']

0      0.00
1     -1.00
2     -2.00
3     -2.00
4     -3.00
5      1.00
6      1.00
7      1.00
8      0.00
9      0.00
10    14.72
11    13.72
12    12.72
13    14.72
14    14.72
15    14.72
16    14.72
17    14.72
18    13.72
19    12.72
20    11.72
21    11.72
22    11.72
23    11.72
24    10.72
25    10.72
26    10.72
27    10.72
28    10.72
29    12.80
30    12.80
31    12.80
32    11.80
33    11.80
34    10.80
35     9.80
36     9.80
37     9.80
38     9.80
39     8.80
40     8.80
41    12.72
42    10.72
43    10.72
Name: running_stake_points_back_win, dtype: float64

In [79]:
p['running_stake_points_back_win']

0      0.00
1     -1.00
2     -2.00
3     -2.00
4     -3.00
5      1.00
6      1.00
7      1.00
8      0.00
9      0.00
10    14.72
11    13.72
12    12.72
13    14.72
14    14.72
15    14.72
16    14.72
17    14.72
18    13.72
19    12.72
20    11.72
21    11.72
22    11.72
23    11.72
24    10.72
25    10.72
26    10.72
27    10.72
28    10.72
29    12.80
30    12.80
31    12.80
32    11.80
33    11.80
34    10.80
35     9.80
36     9.80
37     9.80
38     9.80
39     8.80
40     8.80
41    12.72
42    10.72
43    10.72
Name: running_stake_points_back_win, dtype: float64

In [36]:
df_processed[[
    "profit_loss", "profit_loss_stake_points", "running_total_back_win", "running_stake_points_back_win"
]]

,profit_loss,profit_loss_stake_points,running_total_back_win,running_stake_points_back_win
0,0.80,0.80,0.00,0.0
1,-1.00,-1.00,-1.00,1.0
2,-1.00,-1.00,-2.00,2.0
3,-1.00,-1.00,-2.00,2.0
4,-1.00,-1.00,-3.00,3.0
5,4.00,4.00,1.00,4.0
6,-2.00,-1.00,1.00,4.0
7,0.80,0.80,1.00,4.0
8,-1.00,-1.00,0.00,5.0
9,-0.80,-1.00,0.00,5.0
